In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

env_variables_names = ["TAVILY_API_KEY", "GOOGLE_API_KEY"]
for env_variable in env_variables_names:
    if env_variable not in os.environ:
        raise Exception(f"{env_variable} env variable should be defined")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [3]:
from langchain_tavily import TavilySearch

tavily_tool = TavilySearch(
                max_results=5,
                topic="general",
                # include_answer=False,
                # include_raw_content=False,
                # include_images=False,
                # include_image_descriptions=False,
                # search_depth="basic",
                time_range="week",
                # include_domains=None,
                # exclude_domains=None
            )

In [6]:
resp = llm.invoke(
    "Was there a new discovery regarding renewable energies?",
    tools=[tavily_tool],
)
resp

Key 'additionalProperties' is not supported in schema, ignoring


AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "new discoveries renewable energies", "time_range": "week", "topic": "general"}'}, '__gemini_function_call_thought_signatures__': {'3d138b0d-111f-437b-ab14-12320275dd77': 'CuUEAdHtim8/i8yoTW/zGDJhkWQMQCgSpa1B8+x3ySf+ibgNpeENqE0WHgbQ1W90Ih3gBMapQ4JTh+lrSRMamcKnyuJSQge6TNTsO7fJ/NvyOnoWpj2TfyWbbiK9rHtKutKl/bQ9XQoCXcXRYNwsThLXCn4/fhTDU0BlNw3JEJK4/iLUJXsj8d/g50AvEUu2KgBVkjJ7hXVvtrLkD/SoVKRA1b03DAN3hy+bxP9Kmt08UOCEZUPKtSy+GBATOG97Xes2xYvo5Bz3c6N4HG/CLkKsQrqafbf47wyMPQCv/ubtDQKbAyiiS577bgbqCBQtf1NXGen9pJ5amRX8uSiWTLJ6Zx2pUqWjd5YSHGbXwHNjO4K3SHVaUUyVZpXxvQabNW3fCKbl7U1mixaPpPaaA9nySgnO2kYFGfkm+KmofMjqhP3slUnLsmBIga3lua/Zbgkp/f6jJSN0InQ2RrQOPhJQzfWMGqfjgBt2Nc5b4zXn1ERzo6dvxaTYJjjWyTHx/9X8zjN5qErblREP3ttMYwt0o6amZt28sTBBVcKhSCZc6yDO8nIrLorm4SuOg9+0ZVNjIJ78eAfehC8jlvUfrgFPFCDTD4NH3Ztdepx4SrrC7wyR5xn0fiCCOKxUI269zNoFlnlr5ythbszitETNVo26L/khw5h4nK22oZsijPFO6NWKUV55vrhrKwkSg6YGB5EFFZA40Ij40Tui

Based on the screenshots and the output you provided, your code is actually working, but it is incomplete.

Here is the explanation of why this is happening and the code to fix it.

The Problem: Planning vs. Execution
When you run llm.invoke(..., tools=[tavily_tool]), you are asking the LLM to plan a response.

What happened: The LLM analyzed your question ("Was there a new discovery regarding renewable energies?"). It realized it didn't know the answer off the top of its head.

** The Output:** It returned an AIMessage with content='' (empty text) but with a tool_calls list containing tavily_search.

Translation: The LLM is effectively saying: "I need to search for this. Please run the tavily_search function with the query 'new discoveries renewable energies' and give me back the results so I can write the answer."

Your current code stops there. It does not actually execute the search or feed the results back to the LLM.

The Fix: Use an Agent
To make this work automatically (where the code sees the tool call, runs it, and gets the final answer), you need to wrap your LLM and tools in an AgentExecutor or use a pre-built LangGraph agent.

1. Why the "Old" Code Worked (Server-Side Execution)
When you use GenAITool(google_search={}), you are triggering a special native feature of the Gemini API called "Grounding."

How it works: You send the prompt to Google. Google's servers see the Google Search tool, perform the search internally on their side, process the results, and generate the final answer.

The Result: The API sends back the completed answer (with citations). Your Python code didn't have to do any work—it just received the final text.

2. Why the "New" Code Didn't Work (Client-Side Execution)
When you use TavilySearch (or any standard LangChain tool), you are using Tool Calling.

How it works: You send the prompt to the LLM. The LLM sees the tool definition and thinks, "I need to search."

The Result: The LLM sends back a request for you to run the search (a "Tool Call"). It effectively stops and says, "Here is the function name and arguments. Please run this and give me the result."

The Missing Step: Your code stopped there. It received the "request" but didn't have the logic (the Agent) to actually execute the search and feed the results back.

In [8]:
from langchain.agents import create_agent

agent = create_agent(llm, [tavily_tool])

user_input = "Was there a new discovery regarding renewable energies?"
resp = agent.invoke(
    {"messages": [{"role": "user", "content": user_input}]}
)

In [17]:
resp["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'type': 'text', 'text': "Yes, there has been a significant discovery regarding renewable energies. An estimated 18 million tons of lithium have been discovered beneath California's Salton Sea. This discovery is a boost for the geothermal industry, which produces clean, steady renewable energy while also extracting lithium as a byproduct.", 'extras': {'signature': 'CpoOAdHtim8Um8aGU/RO+0hWEMm2B3KFvQXzxRHYLu7P8l0Wo0ghRAQSVRGa136hZeH4+j4xA5sG9BcghF9O+BEiB7Dp/cVyIEk0Cxvfwe/lRXfbIf4rWFBYtDUgzSHD6miGOAMp9C+AMZc+2y+xQ4pyTUsZERRHtY6DXrHQ0JBQXu9IosCCPzGmMETgrotUyAY0BL0rvj4gNy2RF2qXvAd1UPduJGZL2IXDDCk2u0KSk2rsoaKuTYBo9qwQeHNBXno7K3417rQWPUyZrkUokaMQpqZ+zL56LtLMQw8sXn3Kq+6aISxiMvNyM8YfFNDwBxMO+65nc1ne9OGBVUW4RYat9pafYQRTWOwtU6TImp1trxy9KgZ6WD/5hUGZJb0DsRXYLj5dcTbgVSEB/nuUh9bJPVbpTzSSwsVz3AymwdSKZSNDgG+AsAW/c1PC2timZhpGVYJc5zyYcEvbIZwWLsnGmAzwRptF/VP4qGxP2hLzjKhtU6mq69Cjo0HEIVZsQvfpoTpwEnHW5l7DKHLWLmmdsb90f492LTJdM